In [33]:
import numpy as np
import ast
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from nltk.translate.bleu_score import corpus_bleu 

#1 - make data generator for validation and training
#2 - run the model and save after each epoch and make the model checkpoint for generator
#3 - make generation after training model
#4 - make evaluation -------------------

# Preparation to image caption

In [2]:
import matplotlib.pyplot as plt 
import tensorflow as tf
import numpy as np
import os 
from PIL import Image
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import Model
from keras.layers import Input,Dense ,GRU,Embedding,Dropout,add,Concatenate,Masking,merge,Merge,LSTM
from keras.applications import VGG16
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint,TensorBoard
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import ast
from keras.models import Sequential
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from os import listdir
from pickle import dump,load
from tensorflow.python.keras.utils import to_categorical,plot_model


In [ ]:
mark_start = 'ssss'
mark_end = 'eeee'
captionsl=[]

In [ ]:
for i in range(0,len(dev_data)):
    for j in range(0,len(tokens_for_Imgsll)):
        if(dev_data[i]== tokens_for_Imgsll[j][0]):
                captionsl.append(tokens_for_Imgsll[j][1]) 
                ImgAddress.append(dev_data[i])    

for i in range(0,len(training_dataImgTxt)):
    for j in range(0,len(tokens_for_Imgsll)):
        if(training_dataImgTxt[i]== tokens_for_Imgsll[j][0]):
            captionsl.append(tokens_for_Imgsll[j][1])
            ImgAddress.append(training_dataImgTxt[i])  
    

# Read file of marked Captions 

In [3]:
#after the load the captions and make marks which helps the model to specify the start of captions and its end
def Load_marked_captions():
    captionTL = []
    captionL = []
    file = open('../ProjectPattern/mark_captions.txt','r')
    file2= open('../ProjectPattern/mark_captions_testing.txt','r')
    captionTL = ast.literal_eval(file2.read())
    
    captionsl  = ast.literal_eval(file.read())
    return captionsl,captionTL
captionsl,captionTL=Load_marked_captions()

# Tokenize the captions to words and mapping words to indexes

In [55]:
#this operartion helps the model to make embedding to one hot encoding to each word
def tokenize(captions):
    token = Tokenizer()
    token.fit_on_texts(captions)
    #token.texts_to_sequences(captions)
    return token

tokenizer=tokenize(captionsl)
captionsEncoded=p.texts_to_sequences(captionsl)
vocab_size=len(p.word_index)+1
vocab_size


7928

In [5]:
#calculate the maxlength sequence to help in padding the sequence of captions
def max_length(captions):
    theLongestCaption = max(captionsl)
    theLongestCaption=theLongestCaption.split(' ')
    return len(theLongestCaption)  
max_len =max_length(captionsl)

max_length(captionTL)

12

# Construct dictionary for Specifying data(Train,Test,Validation) 

In [6]:
photo_feature=load(open('feature.pkl','rb'))
desc_list = open('dic_Captions_cleaned.txt','r')
desc_list=ast.literal_eval(desc_list.read())

file = open('ImgAddress.txt','r')

def load_photo_features(photo,keys):
    
    features = {key.strip('.jpg'):photo[key.strip('.jpg')] for key in keys}
    return features

def load_desc(desc,keys):
     
        descriptions = {key.strip('.jpg'):desc[key] for key in keys}
        return descriptions


# Importing pre-trained model VGG

In [ ]:
def extract_feature(directory):
    image_model = VGG16(include_top=True, weights='imagenet')
    image_model.layers.pop()
    #take the output of last layer of the pre-trained after remove the last layer
    model = Model(inputs=image_model.inputs,outputs=image_model.layers[-1].output)  
    i=0
    features = dict()
    for name in listdir(directory):
        #load image by specify the directory of image + name
        filename = directory+'/'+name
        i=i+1
        print(i)
        img =load_img(filename,target_size=(224,224))
        #convert img to numpy array
        img_array =img_to_array(img)
        #reshape the data for the model 
        img = img_array.reshape(1,img_array.shape[0],img_array.shape[1],img_array.shape[2])
        #preprocess input to the model
        img =preprocess_input(img)
        #extract feature by make prediction use the pretrained model
        feature = image_model.predict(img,verbose=0)
        #extract img_id 
        img_id = name.split('.')[0]
        #store in dic
        features[img_id] = feature
    return features 
directory = '../ProjectPattern/Flickr8k_Dataset/Flicker8k_Dataset'
features =extract_feature(directory)
dump(features,open("feature.pkl","wb"))

# Prepare the text data to define the model

In [7]:
def create_sequences(encodedSeqList,max_length,photo):
    X1 ,Y,X2 = [],[],[]
    for seq in  encodedSeqList:
        for index in range(1,len(seq)):
            #split the data sequence into two word the first_seq take the first word but encoded into number and the second seq take the second word encoded 
            in_seq,out_seq = seq[:index],seq[index]
            
            #the pad sequences return the numpy array of 2d so we make subscript to convert it to 1d numpy array
            for j in range(0,max_length-len(in_seq)):
                in_seq.append(0)
                
            out=[]
            out.append(out_seq)  
            #for j in range(0,max_length-1):
                 #out.append(0)    
            #convert the encoded sequence to binary representation
            out_seq=to_categorical([out],num_classes=vocab_size)[0]
            
            #prepare the photo 
            X1.append(in_seq)
            Y.append(out_seq)
            X2.append(photo)
            return X1,X2,Y
create_sequences(captionsEncoded,12,'')            

([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [''],
 [array([0., 0., 0., ..., 0., 0., 0.])])

 # Define model

In [8]:
def define_model(vocab_size,max_length):
    #feature extractor model Dense_map this layer summarizes the contents in image
    fe  = Sequential()
    fe.add(Dense(256,input_shape=(1,1000),activation='relu'))
    fe.add(Dropout(0.5))
    
    
    #sequence model
    seq_model = Sequential()
    
    seq_model.add(Embedding(vocab_size,256,input_shape=(max_length,)))
    seq_model.add(GRU(256,return_sequences=True))
    seq_model.add(GRU(256,return_sequences=True))
    seq_model.add(GRU(256,return_sequences=True))
   
    #output_model
    output_model = Sequential()
    Merged_out= add([fe.output,seq_model.output])
    decoder1  = Dense(256,activation='relu')(Merged_out)
    output =  Dense(vocab_size,activation='softmax')(decoder1)
    model = Model(inputs=[fe.input,seq_model.input],outputs=output)
    model.compile(loss='categorical_crossentropy',optimizer='Adam')
    return model
model=define_model(vocab_size,max_len)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
embedding_1_input (InputLayer)  (None, 12)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 12, 256)      2029568     embedding_1_input[0][0]          
__________________________________________________________________________________________________
dense_1_input (InputLayer)      (None, 1, 1000)      0                                            
__________________________________________________________________________________________________
gru_1 (GRU)                     (None, 12, 256)      393984      embedding_1[0][0]                
__________________________________________________________________________________________________
dense_1 (D

In [29]:
def define_model_2(vocab_size,max_length):
    #feature extractor model Dense_map this layer summarizes the contents in image
    input1= Input(shape=(1000,))
    fe1=Dropout(0.5)(input1)
    fe2=Dense(256,activation='relu')(fe1)
    
    
    
    #sequence model
    
    input2 =Input(shape=(max_length,))
    se1 = Embedding(vocab_size,256,)(input2)
    se2=  Dropout(0.5)(se1)
    se3=  LSTM(256)(se2)
    

    #output_model
    output_model = Sequential()
    decoder1= add([fe2,se3])
    decoder2  = Dense(256,activation='relu')(decoder1)
    output =  Dense(vocab_size,activation='softmax')(decoder2)
    model = Model(inputs=[input1,input2],outputs=output)
    model.compile(loss='categorical_crossentropy',optimizer='Adam')
    return model
model2=define_model_2(vocab_size,max_len)
model2.summary()
#iV=iV.reshape(iV.shape[0],iV.shape[2])
#iT=iT.reshape(iT.shape[0],iT.shape[2])
filepath= 'Lstm-weights.{epoch:04d}-{val_loss:.4f}.hdf5'
checkpoint=ModelCheckpoint(filepath,monitor='val_loss',mode='min',save_best_only=True) 
#tensor=TensorBoard(log_dir='../ProjectPattern/Saving_Model/TensorboardLogs/', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
    
model2.fit([iT,sT],oT,validation_data=([iV,sV],oV),callbacks=[checkpoint],epochs=20)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           (None, 12)           0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 12, 256)      2029568     input_26[0][0]                   
__________________________________________________________________________________________________
dropout_26 (Dropout)            (None, 1000)         0           input_25[0][0]                   
__________________________________________________________________________________________________
dropout_27

# Data generator 

In [ ]:
captionFile = open('l2_captions.txt','r')
dic = open('dic_Captions.txt','r')
photo_feature=load(open('feature.pkl','rb'))

def data_generator(dictionary,train_feature,tokenizer,max_len):

    for k in range(0,len(photo_feature)):
        photo = photo_feature[list(photo_feature.keys())[k]]
        tokenizedList = tokenizer.fit_on_texts(dictionary[list(dictionary.keys())[k]])
        encodedSeqList= tokenizer.texts_to_sequences(dictionary[list(dictionary.keys())[k]])
        in_seq,in_img,out_word=create_sequences(encodedSeqList,max_len,photo)
        yield [[in_img,in_seq],out_word]
        
write = open('dic_Captions_cleaned.txt','r')
dictionary =ast.literal_eval(write.read())
tokenized_dic =tokenize(dictionary)

Tokenize = Tokenizer()


In [22]:

def generate_the_data(photo_features,dicDesc):
    seq,img,out = [],[],[]
    tokenizer =Tokenizer()
    for k in range(0,len(dicDesc)):
        photo = photo_feature[list(photo_feature.keys())[k]]
        tokenizedList = tokenizer.fit_on_texts(dicDesc[list(dicDesc.keys())[k]])
        encodedSeqList= tokenizer.texts_to_sequences(dicDesc[list(dicDesc.keys())[k]])
        in_seq,in_img,out_word=create_sequences(encodedSeqList,max_len,photo)
        seq+=in_seq
        img+=in_img
        out+=out_word
    return np.array(seq),np.array(img),np.array(out)

# Prepare  to train captions and images

In [13]:
#Load the img Address
file = open('img_Address_Training.txt','r')
listTrainImg = ast.literal_eval(file.read())

#using load photo to load the feature of the training dataset images
dicFeatureTrain =load_photo_features(photo_feature,listTrainImg[0:-1])
dicDescTrain = load_desc(desc_list,listTrainImg[0:-1])
sT,iT,oT=generate_the_data(dicFeatureTrain,dicDescTrain)

# Prepare  the captions and images of test set

In [35]:
file = open('Img_Address_Test_Image.txt','r')
file2 = open('mark_captions_testing_dic.txt','r')

captionsTesting=ast.literal_eval(file2.read())
listTestImg = ast.literal_eval(file.read())
dicFeatureTest=load_photo_features(photo_feature,listTestImg[0:-1])
dicDescTest = load_desc(captionsTesting,listTestImg[0:-1])
#sTest,iTest,oTest=generate_the_data(dicFeatureTest,dicDescTest)

In [ ]:
def load_desc_dic(captionsTesting,ImgAddress):
    listImg = []
    l=0
    k=5
    dic ={}
    for j in range(0,len(ImgAddress)):
        for i in range(l,k):
            listImg.append(captionsTesting[i])
        
        dic[ImgAddress[j]]  = listImg
        listImg=[]
    return dic    
     

# Prepare the validation or dev images and their captions

In [23]:
#file = open('../ProjectPattern/Flickr8k_text/Flickr_8k.devImages.txt','r')
#listdevImg=file.read().split('\n')
file = open('Img_Address_devval_Image.txt','r')
listValImg = ast.literal_eval(file.read())
dicFeatureVal=load_photo_features(photo_feature,listValImg[0:-1])
dicDescVal = load_desc(desc_list,listValImg[0:-1])
sV,iV,oV=generate_the_data(dicFeatureVal,dicDescVal)

In [ ]:
next(generatorTrain)[0][1].reshap


# Model running 

In [ ]:
epoch = 100
#the batch_size / numberofsamples
filepath= 'weights.{epoch:04d}-{val_loss:.4f}.hdf5'
steps =2
tokenizer = Tokenizer()
generatorTrain = data_generator(dicDescTrain,dicFeatureTrain,tokenizer,max_len)
generatorVal   = data_generator(dicDescVal,dicFeatureTest,tokenizer,max_len)
model.fit_generator(generatorTrain,steps_per_epoch=1,verbose=1,epochs=100,validation_data=generatorVal,validation_steps=steps)
checkpoint=ModelCheckpoint(filepath,monitor='val_loss',mode='min',save_best_only=True) 
TensorBoard(log_dir='../ProjectPattern/Saving_Model/TensorboardLogs/', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
    

In [ ]:
epoch = 100
#the batch_size / numberofsamples
filepath= 'weights.{epoch:02d}-{val_loss:.2f}.hdf5'
steps =2
tokenizer = Tokenizer()
generatorTrain = data_generator_out(dicDescTrain,dicFeatureTrain,tokenizer,max_len)
generatorVal   = data_generator_out(dicDescVal,dicFeatureTest,tokenizer,max_len)
#ModelCheckpoint(filepath,monitor='val_loss',mode='min') 
#TensorBoard(log_dir='../ProjectPattern/Saving_Model/TensorboardLogs/', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
    

In [ ]:
i=i.reshape(6000,1000)
model2.fit([i,s],o,validation_data=)


dict_items([('a', 1), ('ssss', 2), ('eeee', 3), ('s', 4), ('in', 5), ('the', 6), ('on', 7), ('is', 8), ('and', 9), ('dog', 10), ('with', 11), ('man', 12), ('of', 13), ('two', 14), ('white', 15), ('black', 16), ('boy', 17), ('are', 18), ('woman', 19), ('girl', 20), ('to', 21), ('wearing', 22), ('people', 23), ('at', 24), ('water', 25), ('red', 26), ('brown', 27), ('young', 28), ('an', 29), ('his', 30), ('blue', 31), ('dogs', 32), ('running', 33), ('through', 34), ('playing', 35), ('shirt', 36), ('while', 37), ('down', 38), ('ball', 39), ('standing', 40), ('little', 41), ('grass', 42), ('snow', 43), ('child', 44), ('person', 45), ('jumping', 46), ('over', 47), ('three', 48), ('sitting', 49), ('front', 50), ('holding', 51), ('field', 52), ('up', 53), ('small', 54), ('large', 55), ('by', 56), ('green', 57), ('yellow', 58), ('one', 59), ('group', 60), ('walking', 61), ('children', 62), ('her', 63), ('men', 64), ('into', 65), ('air', 66), ('near', 67), ('mouth', 68), ('beach', 69), ('jumps',

# Generate Description

In [118]:
def word_for_id(integer,tokenizer):
    for word,index in tokenizer.word_index.items():
        if index == integer:
            return word
    
    return None


In [120]:
def generation_desc(model,photo,tokenizer,max_len):
    in_text = 'ssss'    
    for i in range(0,max_len):  
        tokenizer.fit_on_texts(in_text)
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        seq_pad = pad_sequences([sequence],maxlen=max_len)
        y_hat=model2.predict([np.array(photo),np.array(seq_pad)])
        #count the ocurrence of word in the ouput list of list
        y_hat = np.argmax(y_hat)
        
        word = word_for_id(y_hat,tokenizer)
        
        if word == None :
            break
            
        in_text +=' '+word
        if word == 'eeee':
            break
    return in_text        


# Evaluate Model

In [121]:
def evaluate(model,descsTest,photoFeature,tokenizer,max_len):
    yHat,yActual = [],[]
    for key,desc_List in descsTest.items():
        y_hat = generation_desc(model
                                ,photoFeature[key],
                                tokenizer,max_len)
        
        yHat.append(y_hat.split())
     
        theOriginal = [d.split() for d in desc_List]
        yActual.append(theOriginal)
    print('BLEU-1: %f' % corpus_bleu(yActual, yHat, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(yActual, yHat, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(yActual, yHat, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(yActual, yHat, weights=(0.25, 0.25, 0.25, 0.25)))
evaluate(model2,dicDescTest,dicFeatureTest,tokenizer,12)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU-1: 0.135954
BLEU-2: 0.033037
BLEU-3: 0.128764
BLEU-4: 0.180921


In [ ]:
#s=np.array(seq)
#i=np.array(img)
#o=np.array(out)
#data=np.array([[s,i],o])

In [ ]:
#o.shape

In [ ]:
pad_sequences(['2'],maxlen=max_length)[0] 